In [5]:
import face_recognition
import cv2
import numpy as np
import sqlite3
import datetime
import os
import re
import socket
import mediapipe as mp

# === Socket Setup ===
server_ip = '192.168.135.84'  # Change as needed
port = 12345
server_socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
server_socket.bind((server_ip, port))
server_socket.listen(1)
print(f"[SERVER] Waiting for ESP32 to connect at {server_ip}:{port}...")

client_socket, addr = server_socket.accept()
print(f"[SERVER] ESP32 connected from {addr}")

# === SQLite DB Setup ===
conn = sqlite3.connect('log.db', check_same_thread=False)
cursor = conn.cursor()

# Ensure table exists
cursor.execute('''
CREATE TABLE IF NOT EXISTS log (
    name TEXT,
    date TEXT,
    login_time TEXT,
    logout_time TEXT
)
''')
conn.commit()

# === Load Known Faces ===
known_face_encodings = []
known_face_names = []

for file in os.listdir("known_faces"):
    if file.endswith(".jpg"):
        name = re.split(r'\d+', file)[0].capitalize()
        img = face_recognition.load_image_file(f"known_faces/{file}")
        enc = face_recognition.face_encodings(img)[0]
        known_face_encodings.append(enc)
        known_face_names.append(name)

# === Webcam Setup ===
cap = cv2.VideoCapture(0)
recognized_names = {}
threshold_intensity = 100  # Light ON if below this
latest_temp = None
light_status = "Light:OFF"
fan_status = "Fan:OFF"  # Persistent fan status

# === Gesture Recognition ===
mp_hands = mp.solutions.hands
hands = mp_hands.Hands(min_detection_confidence=0.7, min_tracking_confidence=0.7)

try:
    while True:
        ret, frame = cap.read()
        rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)  # FIXED LINE
        locations = face_recognition.face_locations(rgb_frame)
        encodings = face_recognition.face_encodings(rgb_frame, locations)

        now = datetime.datetime.now()
        date = now.strftime("%Y-%m-%d")
        time_now = now.strftime("%H:%M:%S")
        seen_this_frame = set()

        # === Light Intensity Detection ===
        intensity = np.mean(gray_frame)
        light_status = "Light:ON" if intensity < threshold_intensity else "Light:OFF"

        # === Gesture Recognition for Fan Control ===
        results = hands.process(rgb_frame)

        if results.multi_hand_landmarks:
            for hand_landmarks in results.multi_hand_landmarks:
                x_min = min([lm.x for lm in hand_landmarks.landmark])
                x_max = max([lm.x for lm in hand_landmarks.landmark])
                y_min = min([lm.y for lm in hand_landmarks.landmark])
                y_max = max([lm.y for lm in hand_landmarks.landmark])

                palm_open = (x_max - x_min) > 0.3 and (y_max - y_min) > 0.3

                if palm_open:
                    fan_status = "Fan:ON"  # Turn ON and keep ON
                else:
                    fan_status = "Fan:OFF"  # Turn OFF only when a closed palm is detected

        # === Face Recognition ===
        for loc, enc in zip(locations, encodings):
            matches = face_recognition.compare_faces(known_face_encodings, enc, tolerance=0.5)
            name = "Unknown"

            if True in matches:
                face_distances = face_recognition.face_distance(known_face_encodings, enc)
                best_match_idx = np.argmin(face_distances)

                if face_distances[best_match_idx] < 0.5:  # Ensure it's a confident match
                    name = known_face_names[best_match_idx]

            seen_this_frame.add(name)

            if name == "Unknown":
                recognized_names[name] = now  # Track unknown but do not log it
            elif name not in recognized_names:
                recognized_names[name] = now
                cursor.execute("INSERT INTO log (name, date, login_time) VALUES (?, ?, ?)",
                               (name, date, time_now))
                conn.commit()

            # Draw bounding box
            top, right, bottom, left = loc
            cv2.rectangle(frame, (left, top), (right, bottom), (0, 255, 0), 2)
            cv2.putText(frame, name, (left, top - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (255, 0, 0), 2)

        # === Logout Handling ===
        logged_names = list(recognized_names.keys())

        if not seen_this_frame and logged_names:
            for person in logged_names:
                if person != "Unknown":
                    cursor.execute("UPDATE log SET logout_time = ? WHERE name = ? AND date = ? AND logout_time IS NULL",
                                   (time_now, person, date))
                    conn.commit()
            recognized_names.clear()  # Reset tracking when no face is detected

        # === Send Commands to ESP32 Only When a Known Face is Detected ===
        if seen_this_frame and any(name != "Unknown" for name in seen_this_frame):
            msg = f"{light_status}, {fan_status}\n"
            client_socket.sendall(msg.encode('utf-8'))
            print(f"[SERVER] Sent to ESP32: {msg.strip()}")

        # Display status on frame
        cv2.putText(frame, light_status, (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 255), 2)
        cv2.putText(frame, fan_status, (10, 60), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 255), 2)

        cv2.imshow("Cabin Monitor", frame)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

except KeyboardInterrupt:
    print("[INFO] Server stopped manually.")

cap.release()
cv2.destroyAllWindows()
conn.close()
client_socket.close()
server_socket.close()
print("[INFO] All connections closed.")


[SERVER] Waiting for ESP32 to connect at 192.168.135.84:12345...
[SERVER] ESP32 connected from ('192.168.135.120', 61363)
[SERVER] Sent to ESP32: Light:OFF, Fan:OFF
[SERVER] Sent to ESP32: Light:OFF, Fan:OFF
[SERVER] Sent to ESP32: Light:OFF, Fan:OFF
[SERVER] Sent to ESP32: Light:OFF, Fan:OFF
[SERVER] Sent to ESP32: Light:OFF, Fan:OFF
[SERVER] Sent to ESP32: Light:OFF, Fan:OFF
[SERVER] Sent to ESP32: Light:OFF, Fan:OFF
[SERVER] Sent to ESP32: Light:OFF, Fan:OFF
[SERVER] Sent to ESP32: Light:OFF, Fan:OFF
[SERVER] Sent to ESP32: Light:OFF, Fan:OFF
[SERVER] Sent to ESP32: Light:OFF, Fan:OFF
[SERVER] Sent to ESP32: Light:OFF, Fan:OFF
[SERVER] Sent to ESP32: Light:OFF, Fan:OFF
[SERVER] Sent to ESP32: Light:OFF, Fan:OFF
[SERVER] Sent to ESP32: Light:OFF, Fan:OFF
[SERVER] Sent to ESP32: Light:OFF, Fan:OFF
[SERVER] Sent to ESP32: Light:OFF, Fan:OFF
[SERVER] Sent to ESP32: Light:OFF, Fan:OFF
[SERVER] Sent to ESP32: Light:OFF, Fan:OFF
[SERVER] Sent to ESP32: Light:OFF, Fan:OFF
[SERVER] Sent to E

In [ ]:
from flask import Flask, render_template_string, request
import sqlite3
from werkzeug.serving import run_simple

app = Flask(__name__)

# === HTML Templates ===
index_html = """
<!DOCTYPE html>
<html>
<head><title>Cabin Monitor</title></head>
<body>
    <h1>Select a Person</h1>
    <form action="/logs" method="post">
        <select name="person">
            {% for name in names %}
                <option value="{{ name }}">{{ name }}</option>
            {% endfor %}
        </select>
        <button type="submit">View Logs</button>
    </form>
</body>
</html>
"""

logs_html = """
<!DOCTYPE html>
<html>
<head><title>{{ name }}'s Logs</title></head>
<body>
    <h1>Logs for {{ name }}</h1>
    <table border="1">
        <tr><th>Date</th><th>Login</th><th>Logout</th></tr>
        {% for log in logs %}
        <tr>
            <td>{{ log[0] }}</td>
            <td>{{ log[1] }}</td>
            <td>{{ log[2] if log[2] else 'Still Logged In' }}</td>
        </tr>
        {% endfor %}
    </table>
    <br>
    <a href="/">Back</a>
</body>
</html>
"""

def get_names():
    """Fetch distinct names except 'Unknown' from the log database."""
    with sqlite3.connect('log.db') as conn:
        cursor = conn.cursor()
        cursor.execute("SELECT DISTINCT name FROM log WHERE name != 'Unknown'")
        return [row[0] for row in cursor.fetchall()]

@app.route('/')
def index():
    """Homepage to select a person and view logs."""
    return render_template_string(index_html, names=get_names())

@app.route('/logs', methods=['POST'])
def logs():
    """Show logs of a selected person, displaying logout only if they logged out."""
    selected = request.form['person']
    
    with sqlite3.connect('log.db') as conn:
        cursor = conn.cursor()
        cursor.execute("""
            SELECT date, login_time, 
                   CASE 
                       WHEN logout_time IS NULL THEN 'Still Logged In' 
                       ELSE logout_time 
                   END 
            FROM log 
            WHERE name=? 
            ORDER BY date DESC
        """, (selected,))
        logs = cursor.fetchall()

    return render_template_string(logs_html, name=selected, logs=logs)

# Run Flask using werkzeug (better for Jupyter)
run_simple("0.0.0.0", 5000, app, use_reloader=False)


 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5000
 * Running on http://192.168.111.84:5000
Press CTRL+C to quit
192.168.111.84 - - [28/Mar/2025 16:32:49] "GET / HTTP/1.1" 200 -
192.168.111.84 - - [28/Mar/2025 16:32:51] "POST /logs HTTP/1.1" 200 -


In [1]:
import sqlite3

# Connect to the database
conn = sqlite3.connect('log.db')
cursor = conn.cursor()

# Clear the log table
cursor.execute("DELETE FROM log")
conn.commit()

# Close connection
conn.close()

print("✅ Log data cleared successfully!")


✅ Log data cleared successfully!
